In [4]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
dir <- "environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/comp_trees"
suppressPackageStartupMessages(source("tree_helper_function.R"))
load("environment/brain_sim_nodtu/mode=gc_bias/mapDf.RData")
load(file.path(dir, "treeCons.RData"))
load(file.path(dir, "yAggConsNS.RData"))
load(file.path(dir, "treeCor.RData"))
load(file.path(dir, "yAggCorNS.RData"))
load(file.path(dir, "treeCons0.RData"))
load(file.path(dir, "yAggConsNS0.RData"))
load(file.path(dir, "treeMeanInf0.RData"))
load(file.path(dir, "yAggMeanInfNS0.RData"))
load(file.path(dir, "treeThr.RData"))
load(file.path(dir, "yAggThrNS.RData"))
suppressPackageStartupMessages(source("brain_simulation_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/tree_analysis/tree_filter.R"))

In [6]:
dfInf <- data.frame(matrix(nrow=0, ncol=4))
colnames(dfInf) <- c("Type", "Value", "nCuts", "gamma")

l <- length(treeCons$tip)
trees <- list(treeCons, treeCons0, treeMeanInf0, treeCor, treeThr)
names(trees) <- c("ConsFilt", "Cons", "Mean", "AC", "ConsFiltES")
yAgg <- list(yAggConsNS, yAggConsNS0, yAggMeanInfNS0, yAggCorNS, yAggThrNS)
names(yAgg) <- c("ConsFilt", "Cons", "Mean", "AC", "ConsFiltES")

gammas <- c(0.05,0.1,0.5,1,5)
dCutsDInf <- vector(mode = "list", length=length(gammas))
names(dCutsDInf) <- paste("gamma=", gammas, sep="")
for(j in seq_along(dCutsDInf)) {
    gamma <- gammas[j]
    dCutsDInf[[j]] <- vector(mode="list", length(trees))
    names(dCutsDInf[[j]]) <- names(trees)
    for(i in seq(trees)) {
        globArr <- rep(-100, l+trees[[i]]$Nnode)
        descL <- sapply(Descendants(trees[[i]], seq(l+trees[[i]]$Nnode)), length)
        infReps <- assays(yAgg[[i]])[grep("infRep", assayNames(yAgg[[i]]))]
        infReps <- abind::abind(as.list(infReps), along = 3)
        obj <- mcols(yAgg[[i]])[["meanInfRV"]] + gamma*node.depth(trees[[i]],2)
        val <- findOptSum(trees[[i]], obj, l+1, descL)

        gc()
        dCutsDInf[[j]][[i]] <- findCuts(trees[[i]], globArr, obj, l+1, descL)
        gc()
        dfInf <- rbind(dfInf, data.frame(Type = names(trees)[i], Value = val,
               nCuts = length(dCutsDInf[[j]][[i]]), gamma=gamma))
    }
}
print(dfInf)
# save(dfInf, file=file.path(dir, "dfInf.RData"))
# save(dCutsDInf, file=file.path(dir, "dCutsDInf.RData"))

In [7]:
load(file.path(dir, "dfInf.RData"))
load(file.path(dir, "dCutsDInf.RData"))

In [11]:
dCutsDInf[[6]] <- vector(mode="list", length(trees))
names(dCutsDInf[[6]]) <- names(trees)
for(j in seq(2,6))
    dCutsDInf[[j]] <- dCutsDInf[[j-1]]

In [19]:
length(dCutsDInf[[1]][["ConsInd"]])
length(dCutsDInf[[2]][["ConsInd"]])

[1] 83114

[1] 83114

In [3]:
gamma <- 0.05
j <- 1
for(i in seq(trees)) {
    globArr <- rep(-100, l+trees[[i]]$Nnode)
    descL <- sapply(Descendants(trees[[i]], seq(l+trees[[i]]$Nnode)), length)
    infReps <- assays(yAgg[[i]])[grep("infRep", assayNames(yAgg[[i]]))]
    infReps <- abind::abind(as.list(infReps), along = 3)
    obj <- mcols(yAgg[[i]])[["meanInfRV"]] + gamma*node.depth(trees[[i]],2)
    val <- findOptSum(trees[[i]], obj, l+1, descL)

    gc()
    dCutsDInf[[j]][[i]] <- findCuts(trees[[i]], globArr, obj, l+1, descL)
    gc()
    dfInf <- rbind(dfInf, data.frame(Type = names(trees)[i], Value = val,
           nCuts = length(dCutsDInf[[j]][[i]]), gamma=gamma))
}



,Type,Value,nCuts,gamma
,<chr>,<dbl>,<int>,<dbl>
1,ConsInd,71432.19,83114,0.1
2,Cons,71612.51,89247,0.1
3,Mean,72945.97,90209,0.1
4,AC,124265.98,86551,0.1
5,ConsRedInd,124014.10,122235,0.1
6,ConsInd,193261.57,112795,0.5
